<a href="https://colab.research.google.com/github/saishshinde15/Langchain_RAG_BOT/blob/main/Custom_agent_with_custom_prompt_SAAS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain langgraph langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.5/113.5 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00


In [2]:
%pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.5 MB/s eta 0:00:00


In [3]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [4]:
from langchain_groq import ChatGroq

llm = ChatGroq(model_name='llama3-8b-8192',temperature=0.2,max_tokens=1024)

In [5]:
%pip install -qU "langchain-community>=0.2.11" tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.5 MB/s eta 0:00:00


In [6]:
import getpass
import os

if not os.environ.get("TAVILY_API_KEY"):
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Tavily API key:\n")

Tavily API key:
··········


In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(max_results=3,search_depth="advanced")

In [10]:
%pip install --upgrade --quiet  langchain-community arxiv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 6.6 MB/s eta 0:00:00


In [11]:
from langchain_community.utilities.arxiv import ArxivAPIWrapper

In [12]:
arxiv = ArxivAPIWrapper(
    top_k_results=5,
    sort_by="relevance",
    sort_order="descending"
)

In [81]:
import getpass
import os
from langchain_groq import ChatGroq
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.utilities.arxiv import ArxivAPIWrapper

# ... (Your existing code) ...

def rank_research_idea(idea):
    """Ranks a research idea on a scale of 0-10 based on novelty, potential impact, and feasibility."""

    # Combine information from multiple sources.
    try:
        web_search_results = web_search_tool.run(f"research on {idea}")  # Use your web search tool
        arxiv_results = arxiv.run(idea)
        combined_info = f"Web search results: {web_search_results}\nArxiv results: {arxiv_results}"
    except Exception as e:
        print("Error fetching information:", e)
        combined_info = ""

    prompt = f"""
    Rank the following research idea on a scale of 0-10, considering novelty, potential impact, and feasibility:

    Research Idea: {idea}
    Supporting Information: {combined_info}

    Provide a brief explanation for the score.
    """

    try:
        response = llm.predict(prompt)  # Use your LLM
        print(response)  # Directly print for now

        # Extract the score (this needs to be refined for robustness)
        lines = response.split('\n')
        for line in lines:
            if "Score:" in line:
                try:
                    score = int(line.split("Score:")[1].strip())
                    if 0 <= score <= 10:
                        return score, response
                    else:
                        print("Warning: Score outside 0-10 range. Returning 0.")
                        return 0, response
                except ValueError:
                    print("Warning: Could not extract a valid score. Returning 0.")
                    return 0, response

        print("Warning: No score found in the response. Returning 0.")
        return 0, response
    except Exception as e:
        print("Error during LLM processing:", e)
        return 0, "Error processing the idea"



# Example usage
research_idea = input("Enter your research idea: ")
score, explanation = rank_research_idea(research_idea)

print(f"Research Idea: {research_idea}")
print(f"Score: {score}")
print(f"Explanation: {explanation}")

Enter your research idea: seed using llm


<ipython-input-81-c06364770f1e>:33: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(prompt)  # Use your LLM


I would rank the research idea a 7 out of 10, considering novelty, potential impact, and feasibility.

Novelty: The research idea explores the application of Large Language Models (LLMs) in various domains, including data curation, image tokenization, and code generation. While LLMs have been widely used in natural language processing tasks, their application in these domains is relatively new and innovative.

Potential Impact: The potential impact of this research is significant, as it can enable the development of more efficient and effective data curation tools, improve image-to-text and text-to-image generation capabilities, and enhance code generation in specific scenarios. These applications have the potential to revolutionize various industries, such as data science, computer vision, and software development.

Feasibility: The feasibility of this research is high, as the authors have already demonstrated the effectiveness of their approach in various experiments. The use of LLMs

In [14]:
from langchain.tools import Tool

In [82]:
tools = [
    Tool(
        name="Tavily Search",
        description="Useful for searching current information and real-time results",
        func=web_search_tool .run
    ),
    Tool(
        name="ArXiv Search",
        description="Useful for searching academic papers and research articles",
        func=arxiv.run
    ),
    Tool(
        name="Rank Research Idea",
        description="Useful for ranking research ideas on a scale of 0-10",
        func=rank_research_idea
    )
]

In [83]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")

In [84]:
from langchain.agents import AgentExecutor, create_react_agent, initialize_agent

In [85]:
agent = initialize_agent(
    agent='conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
)

In [86]:
agent.run(input="I have a idea for a research paper that uses llm for seed colassifications.Can you help me?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Rank Research Idea
Action Input: "LLM for seed classifications"I would rank the research idea "LLM for seed classifications" as a 6 out of 10, considering novelty, potential impact, and feasibility.

Novelty: The idea of using LLMs for seed classifications is not entirely new, as seen in the supporting information and Arxiv results. However, the specific application of LLMs for seed classifications is still an area of ongoing research, and there is room for innovation and improvement.

Potential Impact: The potential impact of this research idea is moderate. Seed classifications are an important task in various domains, such as text classification, sentiment analysis, and topic modeling. By developing an LLM-based approach for seed classifications, the research could potentially improve the accuracy and efficiency of these tasks. However, the impact may be limited by the existing research in this area

"That's a great research idea! Using LLMs for seed classifications could potentially improve the accuracy and efficiency of various tasks. I'd be happy to help you explore this idea further. What specific aspects of the research idea would you like to discuss or work on next?"

In [ ]:
## Custom agent with custom prompt

In [89]:
from langchain.agents import AgentType, Tool, initialize_agent
from langchain.memory import ConversationBufferMemory

# Assuming 'llm', 'tools', and 'memory' are defined as in the previous code

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True,
    max_iterations=3,
    memory=memory,
    agent_kwargs={"prefix": """You are a helpful AI research assistant.  Always follow the following steps:
1. **Analyze the user's request carefully.**  Understand the core research question, desired outcomes, and any constraints.
2. **Leverage provided tools.**  Use available tools like Tavily Search and ArXiv Search to gather relevant information.
3. **Rank the idea (always).** If the user asks to rank a research idea, utilize the Rank Research Idea tool.  Follow its output for ranking and explanation.
4. **Synthesize and respond.**  Combine information from all steps to construct a comprehensive and insightful response to the user.  If a research idea is ranked, incorporate the score and justification in your answer.
5. **Maintain conversation history.**  Use your memory to track previous interactions and build upon prior context.  Avoid redundant searches or explanations.
6. **Help the user.** Use the Rank Research Idea tool everytime the user provides with any idea.Then give a short summary what user can do using that idea.

    {chat_history}
    Question: {input}
    Thought:"""}
)

# Example usage (you can replace this with your desired input)
agent.run(input="I have a idea for a research paper that uses llm for seed colassifications.Can you help me?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Rank Research Idea
Action Input: The idea is to use LLMs for seed classificationsI would rank the research idea "Using LLMs for seed classifications" as a 6 out of 10.

Novelty: 4/10 - The idea of using LLMs for seed classifications is not entirely new, as seen in the provided research papers. However, the specific application of LLMs for seed classification has not been extensively explored, so there is some novelty in this idea.

Potential Impact: 7/10 - The potential impact of this research idea is significant, as it could lead to more accurate and efficient seed classification methods. This could have important implications for the agricultural industry, particularly in the context of crop breeding and seed quality control.

Feasibility: 8/10 - The feasibility of this research idea is high, as LLMs have already been shown to be effective in various classification tasks. The supporting information 

'Based on the Rank Research Idea tool, I would rank the research idea "Using LLMs for seed classifications" as a 6 out of 10. The idea has some novelty, potential impact, and feasibility, but it\'s not entirely new and has some limitations. \n\nThis research idea has the potential to improve the accuracy and efficiency of seed classifications, which could have significant implications for the agricultural industry. To move forward with this idea, you could explore the following:\n\n* Review existing research papers on LLMs and seed classification to understand the current state of the art and identify potential gaps in the literature.\n* Investigate the specific challenges and limitations of using LLMs for seed classification, such as data quality, labeling, and evaluation metrics.\n* Design and implement experiments to evaluate the performance of LLMs for seed classification, using a variety of datasets and evaluation metrics.\n* Explore potential applications of LLMs for seed classif

In [90]:
agent.run(input="okay , but can you give me some papers that i can uses as refrence for my paper")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: ArXiv Search
Action Input: "LLMs and seed classification" or "natural language processing for seed classification"
Observation: Published: 2021-04-20
Title: Seed Word Selection for Weakly-Supervised Text Classification with Unsupervised Error Estimation
Authors: Yiping Jin, Akshay Bhatia, Dittaya Wanvarie
Summary: Weakly-supervised text classification aims to induce text classifiers from
only a few user-provided seed words. The vast majority of previous work assumes
high-quality seed words are given. However, the expert-annotated seed words are
sometimes non-trivial to come up with. Furthermore, in the weakly-supervised
learning setting, we do not have any labeled document to measure the seed
words' efficacy, making the seed word selection process "a walk in the dark".
In this work, we remove the need for expert-curated seed words by first mining
(noisy) candidate seed words associated with the catego

'I\'ve found some relevant papers on LLMs and seed classification that you can use as references for your paper. Here are the results of the ArXiv Search:\n\n* "Seed Word Selection for Weakly-Supervised Text Classification with Unsupervised Error Estimation" by Yiping Jin, Akshay Bhatia, and Dittaya Wanvarie (2021)\n* "A Benchmark on Extremely Weakly Supervised Text Classification: Reconcile Seed Matching and Prompting Approaches" by Zihan Wang, Tianle Wang, Dheeraj Mekala, and Jingbo Shang (2023)\n* "Debiasing Made State-of-the-art: Revisiting the Simple Seed-based Weak Supervision for Text Classification" by Chengyu Dong, Zihan Wang, and Jingbo Shang (2023)\n\nThese papers explore different approaches to weakly supervised text classification using seed words, including seed word selection, benchmarking, and debiasing. You may find them useful in developing your research idea on using LLMs for seed classifications.\n\nAdditionally, you can also explore other papers on LLMs and natural